# OpenAI-to-Z Challenge: Advanced Analysis

This notebook builds on Checkpoint 1. Our goal is to refine the analysis pipeline to get more meaningful results from the OpenAI vision model.

**Objectives:**
1. Enhance the Digital Terrain Model (DTM) to highlight subtle features.
2. Tile the DTM into smaller, manageable chunks for analysis.
3. Use a more sophisticated prompt to guide the OpenAI model.
4. Process the results and visualize them on an interactive map.

## 1. Setup and Imports

In [1]:
import os
import rasterio
from rasterio.windows import Window
import numpy as np
import openai
from dotenv import load_dotenv
from PIL import Image
import base64
from io import BytesIO
import json
import cv2 # OpenCV for image processing
import geopandas as gpd
import folium
from shapely.geometry import box
from tqdm import tqdm

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = openai.OpenAI()

print("Libraries imported and OpenAI client initialized.")

Libraries imported and OpenAI client initialized.


## 2. Load DTM Data

In [2]:
dtm_raster_path = '/Users/shg/Projects/openai-a-z-challenge/data/raw/TAL_A01_2018/TAL_A01_2018_DTM/TAL01L0001C0002.grd'

try:
    with rasterio.open(dtm_raster_path) as src:
        dtm_profile = src.profile
        dtm_bounds = src.bounds
        dtm_crs = src.crs
        print(f"DTM file loaded successfully: {dtm_raster_path}")
        print(f"  - CRS: {dtm_crs}")
        print(f"  - Bounds: {dtm_bounds}")
except rasterio.errors.RasterioIOError as e:
    print(f"Error loading DTM file: {e}")

DTM file loaded successfully: /Users/shg/Projects/openai-a-z-challenge/data/raw/TAL_A01_2018/TAL_A01_2018_DTM/TAL01L0001C0002.grd
  - CRS: None
  - Bounds: BoundingBox(left=611108.0, bottom=8866715.0, right=612109.0, top=8867716.0)


## 3. Image Enhancement and Tiling

To improve the model's ability to detect subtle features, we will first enhance the contrast of the DTM image using histogram equalization. Then, we'll break the large DTM into smaller, overlapping tiles.

In [3]:
def enhance_image(image_data):
    """Applies histogram equalization to enhance image contrast."""
    # Normalize to 0-255 range for 8-bit image processing
    normalized = cv2.normalize(image_data, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    # Apply histogram equalization
    equalized = cv2.equalizeHist(normalized)
    return equalized

def tile_raster(raster_path, tile_size=(512, 512), overlap=128):
    """Tiles a large raster into smaller, overlapping chunks."""
    tiles = []
    with rasterio.open(raster_path) as src:
        width, height = src.width, src.height
        stride = tile_size[0] - overlap
        for y in range(0, height, stride):
            for x in range(0, width, stride):
                # Define the window for the tile
                window = Window(x, y, tile_size[0], tile_size[1])
                # Read the data, enhance it, and get transform for coordinates
                tile_data = src.read(1, window=window)
                enhanced_tile = enhance_image(tile_data)
                tile_transform = src.window_transform(window)
                tiles.append((enhanced_tile, tile_transform))
    print(f"Generated {len(tiles)} tiles from {raster_path}")
    return tiles

# Generate tiles from our DTM
try:
    dtm_tiles = tile_raster(dtm_raster_path)
except NameError:
    print("DTM file not loaded, skipping tiling.")
    dtm_tiles = []

Generated 9 tiles from /Users/shg/Projects/openai-a-z-challenge/data/raw/TAL_A01_2018/TAL_A01_2018_DTM/TAL01L0001C0002.grd


## 4. Advanced OpenAI API Analysis

Now we'll loop through each enhanced tile and send it to the OpenAI model with a more detailed prompt. We'll ask for a structured JSON output to make the results easier to parse.

In [4]:
def numpy_to_base64(np_array):
    """Converts a NumPy array to a base64 encoded PNG image."""
    img = Image.fromarray(np_array, 'L') # 'L' for grayscale
    buffered = BytesIO()
    img.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

def analyze_tile_with_openai(image_tile):
    """Sends a single tile to the OpenAI API for analysis."""
    base64_image = numpy_to_base64(image_tile)
    
    MODEL = "gpt-4o"
    PROMPT = """You are an expert remote sensing archaeologist. Analyze this enhanced Digital Terrain Model (DTM) image from the Amazon rainforest. Your task is to identify potential pre-Columbian archaeological earthworks (geoglyphs).
    
    Look for the following patterns:
    1.  **Geometric Shapes:** Clear geometric forms such as circles, squares, rectangles, or octagons. These may appear as ditches or embankments.
    2.  **Linear Features:** Long, straight lines or causeways that are not consistent with modern roads or infrastructure.
    
    Ignore modern features like recent deforestation lines, roads, or buildings. Focus only on patterns that could be ancient.
    
    Respond with a JSON object. The object should contain a single key, 'features', which is a list of found objects. Each object in the list should have:
    - 'shape': A string describing the shape (e.g., 'circle', 'rectangle', 'linear_ditch').
    - 'center_coords': The approximate center of the feature as [x, y] coordinates, scaled from 0.0 to 1.0 (top-left is [0,0]).
    - 'confidence': Your confidence level (Low, Medium, High) that this is a potential archaeological feature.
    - 'description': A brief justification for your finding.
    
    If you find no potential features, return an empty list: {"features": []}."""

    try:
        response = client.chat.completions.create(
            model=MODEL,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": PROMPT},
                        {
                            "type": "image_url",
                            "image_url": {"url": f"data:image/png;base64,{base64_image}"}
                        }
                    ]
                }
            ],
            temperature=0.1,
            response_format={"type": "json_object"} # Request JSON output
        )
        return json.loads(response.choices[0].message.content)
    except Exception as e:
        print(f"An error occurred during API call: {e}")
        return None

# Loop through tiles and analyze them
analysis_results = []
if dtm_tiles:
    for i, (tile_image, tile_transform) in enumerate(tqdm(dtm_tiles, desc="Analyzing Tiles")):
        result = analyze_tile_with_openai(tile_image)
        if result and result.get('features'):
            for feature in result['features']:
                analysis_results.append({
                    "tile_index": i,
                    "transform": tile_transform,
                    "feature": feature
                })
print(f"Analysis complete. Found {len(analysis_results)} potential features.")

Analyzing Tiles:   0%|          | 0/9 [00:00<?, ?it/s]

Analyzing Tiles:  11%|█         | 1/9 [00:00<00:05,  1.41it/s]

Analyzing Tiles:  22%|██▏       | 2/9 [00:01<00:04,  1.59it/s]

Analyzing Tiles:  33%|███▎      | 3/9 [00:01<00:03,  1.65it/s]

Analyzing Tiles:  44%|████▍     | 4/9 [00:02<00:03,  1.37it/s]

...

Analyzing Tiles:  78%|███████▊  | 7/9 [00:05<00:01,  1.48it/s]

Analyzing Tiles:  89%|████████▉ | 8/9 [00:05<00:00,  1.40it/s]

Analyzing Tiles: 100%|██████████| 9/9 [00:06<00:00,  1.40it/s]

Analyzing Tiles: 100%|██████████| 9/9 [00:06<00:00,  1.38it/s]

Analysis complete. Found 0 potential features.


## 5. Process and Visualize Results

Finally, we'll convert the analysis results into a GeoDataFrame and plot them on an interactive map using Folium.

In [5]:
def results_to_geodataframe(results, dtm_crs):
    """Converts the list of analysis results to a GeoDataFrame."""
    records = []
    for res in results:
        feature = res['feature']
        transform = res['transform']
        # Get image coordinates from relative 0-1 scale
        img_x = feature['center_coords'][0] * 512
        img_y = feature['center_coords'][1] * 512
        # Convert image coordinates to geographic coordinates
        geo_x, geo_y = rasterio.transform.xy(transform, img_y, img_x)
        
        records.append({
            'geometry': gpd.points_from_xy([geo_x], [geo_y])[0],
            'shape': feature['shape'],
            'confidence': feature['confidence'],
            'description': feature['description']
        })
    
    if not records:
        return None
        
    gdf = gpd.GeoDataFrame(records, crs=dtm_crs)
    return gdf

# Create GeoDataFrame
if analysis_results:
    features_gdf = results_to_geodataframe(analysis_results, dtm_crs)
    if features_gdf is not None:
        # Reproject to WGS84 for Folium map
        features_gdf_wgs84 = features_gdf.to_crs(epsg=4326)

        # Create a map centered on the DTM bounds
        center_y, center_x = (dtm_bounds.top + dtm_bounds.bottom) / 2, (dtm_bounds.left + dtm_bounds.right) / 2
        
        # Need to reproject center point for Folium
        center_point_gdf = gpd.GeoDataFrame([{'geometry': gpd.points_from_xy([center_x], [center_y])[0]}], crs=dtm_crs).to_crs(epsg=4326)
        map_center = [center_point_gdf.geometry.y[0], center_point_gdf.geometry.x[0]]

        m = folium.Map(location=map_center, zoom_start=12, tiles='CartoDB positron')

        # Add detected features to the map
        for _, row in features_gdf_wgs84.iterrows():
            folium.Marker(
                location=[row.geometry.y, row.geometry.x],
                popup=f"<b>Shape:</b> {row['shape']}<br><b>Confidence:</b> {row['confidence']}<br><b>Desc:</b> {row['description']}",
                icon=folium.Icon(color='red', icon='info-sign')
            ).add_to(m)
            
        # Add DTM boundary to map
        bounds_poly = box(dtm_bounds.left, dtm_bounds.bottom, dtm_bounds.right, dtm_bounds.top)
        bounds_gdf = gpd.GeoDataFrame([{'geometry': bounds_poly}], crs=dtm_crs).to_crs(epsg=4326)
        folium.GeoJson(bounds_gdf, style_function=lambda x: {'fillColor': 'blue', 'color': 'blue'}).add_to(m)

        display(m)
    else:
        print("No features were found to visualize.")
else:
    print("No analysis results to display.")

No analysis results to display.
